In [70]:
import pyvisa as pv
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
from fileinput import close
import keyboard
import datetime
import time

In [67]:
def connectToOscil():
    rm = pv.ResourceManager()
    #Get the USB device, e.g. ""
    instruments = rm.list_resources()
    usb = list(filter(lambda x: 'USB' in x, instruments))

    if len(usb) !=1:
        print('Bad instrument list', instruments)
        sys.exit(-1)
    scope = rm.open_resource(usb[0], timeout=1000, chunk_size=1024000)
    return rm, usb, scope

def readDataFromOscil (rm, usb, scope):

    # Create boundaries values for the batch reading data from oscilloscope
    # The maximum batch size for require data type BYTE is 250 000
    # Our oscilloscope memory is 12 000 000 bytes, so there must be 48 batches to read all data
    boundariesRight = np.arange(0, 12250000, 250000) 
    boundariesLeft = np.arange(1, 12000000, 250000)

    # send commands to oscilloscope
    scope.write(":STOP")            # Set the instrument to STOP state (you can only read the waveform data in the internal memory when the oscilloscope is in STOP state)
    scope.write(":WAV:SOUR CHAN1")  # Set the channel source to CH1
    scope.write(":WAV:MODE RAW")    # Set the waveform reading mode to RAW
    scope.write(":WAV:FORM BYTE")   # Set the return format of the waveform data to BYTE

    # Perform the partial reading operation
    for pos in range(47):
        scope.write(":WAV:STAR " + str(boundariesLeft[pos]))        # Set the START point of the reading operation waveform point
        scope.write(":WAV:STOP " + str(boundariesRight[pos+1]))     # Set the STOP point of the reading operation waveform point
        scope.write(":WAV:DATA?")                                   # Read the data from the START waveform point to the STOP waveform point
        
        rawdata = scope.read_raw()                                  # Read the raw data
        rawdata = rawdata[10 :]                                     # Drop the heading
        data = np.frombuffer(rawdata, 'B')                          # Interpret a buffer as a 1-dimensional array. Data type is st to byte = 'B'.
        allData = np.append(data)                                   # Add current block of data to the final array

    return allData


In [86]:
allData = np.array(np.zeros(1))

rm = pv.ResourceManager()
#Get the USB device
instruments = rm.list_resources()
usb = list(filter(lambda x: 'USB' in x, instruments))

if len(usb) !=1:
    print('Bad instrument list', instruments)
    sys.exit(-1)
scope = rm.open_resource(usb[0], timeout=1000, chunk_size=1024000)

# Create boundaries values for the batch reading data from oscilloscope
# The maximum batch size for require data type BYTE is 250 000
# Our oscilloscope memory is 12 000 000 bytes, so there must be 48 batches to read all data
boundariesRight = np.arange(0, 12250000, 250000) 
boundariesLeft = np.arange(1, 12000000, 250000)

while True:

    # try:
    #     if keyboard.is_pressed('s'):

           
    #     elif keyboard.is_pressed('q'):
    #         print('\nYou press "exit". Process teminated.')
    #         sys.exit(0)
    # except: 
    #     print(str(dateNow))
    #     print('Something bad occure')
    #     break

    if keyboard.is_pressed('s'):
        
        dateNow = datetime.datetime.today().strftime("%Y-%m-%d %H.%M.%S")

        # send commands to oscilloscope
        scope.write(":STOP")                 # Set the instrument to STOP state (you can only read the waveform data in the internal memory when the oscilloscope is in STOP state)
        scope.write(":WAV:SOUR CHAN1")       # Set the channel source to CH1
        scope.write(":WAV:POIN:MODE RAW")    # Set the waveform reading mode to RAW
        scope.write(":WAV:FORM BYTE")        # Set the return format of the waveform data to BYTE

        print('1st point reached')

        # Perform the partial reading operation in RAW mode
        for pos in range(24):
            scope.write(":WAV:STAR " + str(boundariesLeft[pos]))        # Set the START point of the reading operation waveform point
            scope.write(":WAV:STOP " + str(boundariesRight[pos+1]))     # Set the STOP point of the reading operation waveform point
            scope.write(":WAV:DATA?")                                   # Read the data from the START waveform point to the STOP waveform point
            
            rawdata = scope.read_raw()                                  # Read the raw data
            rawdata = rawdata[10 :]                                     # Drop the heading
            data = np.frombuffer(rawdata, 'B')                          # Interpret a buffer as a 1-dimensional array. Data type is st to byte = 'B'.
            data = data[1:-1]
            allData = np.append(allData, data)                          # Add current block of data to the final array

        
        print('2nd point reached')

        timeScale = float(scope.query(":TIM:MAIN:SCAL?"))
        timeoffset = float(scope.query(":TIM:OFFS?"))
        voltscale = float(scope.query(":CHAN1:SCAL?"))
        voltoffset = float(scope.query(":CHAN1:OFFS?"))
        yOrigin = float(scope.query(":WAV:YOR?"))
        yRef = float(scope.query(":WAV:YREF?"))
        yInc = float(scope.query(":WAV:YINC?"))

        #allData = allData * -1 + 255
        allData = (allData -yOrigin - yRef) * yInc
        #plt.plot(allData[1:])

        print('3rd point reached')
        
        infoData = ['timeScale = ' + str(timeScale),
                    'timeoffset = ' + str(timeoffset),
                    'voltscale = ' + str(voltscale),
                    'voltoffset = ' + str(voltoffset)]
        df = pd.DataFrame(infoData)
        df2 = pd.DataFrame(allData)
        df = pd.concat([df, df2])
        

        with open('data/' + (str(dateNow) + ' data.txt'), 'w') as f:

            # Write data to file
            f.write(df.to_string())
            f.close
        
        print('4th point reached')

        print((str(dateNow) + ' data.txt <------- Has been saved (͡ᵔ ͜ʖ ͡ᵔ)'))

        scope.write(":RUN")
    elif keyboard.is_pressed('q'):
        print('\nYou press "exit". Process teminated. ')
        sys.exit(0)


    time.sleep(0.08)

1st point reached
2nd point reached
3rd point reached
4th point reached
2022-11-02 16.52.49 data.txt <------- Has been saved (͡ᵔ ͜ʖ ͡ᵔ)

You press "exit". Process teminated. 


SystemExit: 0

c:\Users\iRU\Documents\Шахов\Code\PA-RSCF-22-72-00015-\.venv\lib\site-packages\IPython\core\interactiveshell.py:3386: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
# import csv
# import pandas as pd

# with open('data/test.csv', 'w') as csvFile:
#     fw = csv.writer(csvFile, delimiter =',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
#     csvFile.close

# df = pd.DataFrame(data)
# df = df.drop([df.index[0], df.index[-1]])
# df.plot.line()


